# Comparing Sports Primes: Analysis of Peak Performance Across Major American Sports

This notebook analyzes the "prime" years of athletes across the four major American sports: MLB, NFL, NBA, and NHL.

## Project Overview
We'll examine when athletes reach their peak performance by analyzing historical data from players who started their careers around 2000 and ended around 2015 (year range is just a test range for now)

##Defining "Prime"
The period of time in a sports player's career when they play their best, statically.
<bl>
**Note: There will be outliers with Hall of Famers and players who play longer than average. The average player in all major sports has a significantly shorter career than the average starting lineup player.

## Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sportsipy.mlb.roster import Roster
from sportsipy.mlb.teams import Teams
from sportsipy.mlb.players import Player
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

## MLB Analysis: Hits per At-Bat vs Age

We'll start with MLB data, analyzing batting performance (hits per at-bat) relative to player age for careers spanning 2000-2015.

In [ ]:
# Test sportsipy API connection
print("Testing sportsipy API connection...")

try:
    # Get a sample team to test the API
    teams_2000 = Teams(2000)
    sample_team = teams_2000[0]  # Get first team
    print(f"Successfully connected! Sample team: {sample_team.name}")
    
    # Test player data access
    roster_2000 = Roster(sample_team.abbreviation, 2000)
    sample_player = roster_2000.players[0]
    print(f"Sample player access successful: {sample_player.name}")
    
except Exception as e:
    print(f"API connection failed: {e}")
    print("Please check your internet connection and try again.")

In [ ]:
def get_mlb_player_career_stats(player_name_partial, start_year=2000, end_year=2015):
    """
    Get career statistics for an MLB player across multiple seasons using sportsipy.
    
    Note: sportsipy API structure may vary, so this function includes fallback handling.
    For more reliable data extraction, consider using the sportsipy Player class directly
    with known player IDs.
    """
    career_data = []
    print(f"Searching for player: {player_name_partial} from {start_year} to {end_year}")
    
    # Search through teams and years to find the player
    for year in range(start_year, end_year + 1):
        try:
            teams = Teams(year)
            for team in teams:
                try:
                    roster = Roster(team.abbreviation, year)
                    for player in roster.players:
                        if player_name_partial.lower() in player.name.lower():
                            print(f"Found potential match: {player.name} on {team.name} in {year}")
                            
                            # Get detailed player stats for the specific year
                            player_obj = Player(player.player_id)
                            
                            # Extract relevant stats for the year
                            # Note: sportsipy structure may vary, so we use getattr with defaults
                            season_stats = {
                                'name': player.name,
                                'year': year,
                                'age': getattr(player_obj, 'age', None),
                                'hits': getattr(player_obj, 'hits', 0),
                                'at_bats': getattr(player_obj, 'at_bats', 0),
                                'batting_average': getattr(player_obj, 'batting_average', 0),
                                'games': getattr(player_obj, 'games', 0)
                            }
                            
                            # Only include seasons with meaningful playing time
                            if season_stats['at_bats'] > 50:
                                career_data.append(season_stats)
                                print(f"Added season: {year} - {season_stats['hits']} hits in {season_stats['at_bats']} AB")
                                
                except Exception as e:
                    # Continue searching other teams/players
                    continue
        except Exception as e:
            # Continue to next year
            continue
    
    print(f"Total seasons found: {len(career_data)}")
    return pd.DataFrame(career_data)

# Sample players who had careers spanning our target period
sample_players = ['Derek Jeter', 'Albert Pujols', 'Barry Bonds', 'Alex Rodriguez', 'Manny Ramirez']

print("Function defined. Ready to extract player data using sportsipy.")

## Note on sportsipy API Usage

The sportsipy library provides access to sports data, but the API structure can be complex. For this analysis, we'll use a combination of:

1. **Teams()** - to get team information for a given year
2. **Roster()** - to get player rosters for specific teams
3. **Player()** - to get detailed player statistics

**Important considerations:**
- Player data structure may vary between seasons
- Some players may not have complete statistics for all years
- The API may have rate limits or require internet connectivity
- For production use, consider caching data or using known player IDs

If the API extraction fails, the notebook will fall back to sample data for demonstration purposes.


In [ ]:
# Extract data for a sample player to test our ETL process
print("Starting ETL process for sample MLB players using sportsipy...")

# Start with a simplified approach - get data for one well-known player
try:
    # Try to get Derek Jeter's data using the sportsipy function
    print("Attempting to extract Derek Jeter's career data...")
    jeter_df = get_mlb_player_career_stats('Derek Jeter', 2000, 2015)
    
    if not jeter_df.empty:
        print(f"Successfully extracted {len(jeter_df)} seasons for Derek Jeter")
        print("\nFirst few records:")
        print(jeter_df.head())
        mlb_df = jeter_df
    else:
        print("No data found for Derek Jeter, using sample data...")
        raise Exception("No API data available")
        
except Exception as e:
    print(f"API extraction failed: {e}")
    print("Using sample data for demonstration...")
    
    # Sample data structure for demonstration (fallback when API fails)
    sample_mlb_data = {
        'Derek Jeter': [
            {'year': 2000, 'age': 26, 'hits': 201, 'at_bats': 593, 'games': 148},
            {'year': 2001, 'age': 27, 'hits': 191, 'at_bats': 614, 'games': 150},
            {'year': 2002, 'age': 28, 'hits': 191, 'at_bats': 644, 'games': 157},
            {'year': 2003, 'age': 29, 'hits': 156, 'at_bats': 482, 'games': 119},
            {'year': 2004, 'age': 30, 'hits': 188, 'at_bats': 643, 'games': 154},
            {'year': 2005, 'age': 31, 'hits': 202, 'at_bats': 654, 'games': 159},
            {'year': 2006, 'age': 32, 'hits': 214, 'at_bats': 623, 'games': 153},
            {'year': 2007, 'age': 33, 'hits': 206, 'at_bats': 639, 'games': 156},
            {'year': 2008, 'age': 34, 'hits': 179, 'at_bats': 596, 'games': 150},
            {'year': 2009, 'age': 35, 'hits': 212, 'at_bats': 634, 'games': 153},
            {'year': 2010, 'age': 36, 'hits': 179, 'at_bats': 663, 'games': 157},
            {'year': 2011, 'age': 37, 'hits': 162, 'at_bats': 546, 'games': 131},
            {'year': 2012, 'age': 38, 'hits': 216, 'at_bats': 683, 'games': 159},
            {'year': 2013, 'age': 39, 'hits': 190, 'at_bats': 628, 'games': 157},
            {'year': 2014, 'age': 40, 'hits': 149, 'at_bats': 581, 'games': 145}
        ]
    }
    
    # Convert to DataFrame
    mlb_data_list = []
    for player_name, seasons in sample_mlb_data.items():
        for season in seasons:
            season['name'] = player_name
            season['hits_per_ab'] = season['hits'] / season['at_bats'] if season['at_bats'] > 0 else 0
            season['batting_average'] = season['hits_per_ab']  # These are the same metric
            mlb_data_list.append(season)
    
    mlb_df = pd.DataFrame(mlb_data_list)
    print(f"Created sample DataFrame with {len(mlb_df)} records")
    print("\nFirst few records:")
    print(mlb_df.head())

In [ ]:
# Data Analysis and Visualization
print("Analyzing hits per at-bat vs age relationship...")

# Basic statistics
print("\nDataset Summary:")
print(f"Years covered: {mlb_df['year'].min()} - {mlb_df['year'].max()}")
print(f"Age range: {mlb_df['age'].min()} - {mlb_df['age'].max()}")
print(f"Average hits per at-bat: {mlb_df['hits_per_ab'].mean():.3f}")
print(f"Peak performance (highest hits/AB): {mlb_df['hits_per_ab'].max():.3f} at age {mlb_df.loc[mlb_df['hits_per_ab'].idxmax(), 'age']}")

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Hits per AB vs Age (scatter plot)
axes[0, 0].scatter(mlb_df['age'], mlb_df['hits_per_ab'], alpha=0.7, s=60)
axes[0, 0].set_xlabel('Age')
axes[0, 0].set_ylabel('Hits per At-Bat')
axes[0, 0].set_title('Hits per At-Bat vs Age')
axes[0, 0].grid(True, alpha=0.3)

# Add trend line
z = np.polyfit(mlb_df['age'], mlb_df['hits_per_ab'], 2)
p = np.poly1d(z)
axes[0, 0].plot(sorted(mlb_df['age']), p(sorted(mlb_df['age'])), "r--", alpha=0.8, linewidth=2)

# Plot 2: Performance by age (box plot)
mlb_df.boxplot(column='hits_per_ab', by='age', ax=axes[0, 1])
axes[0, 1].set_title('Performance Distribution by Age')
axes[0, 1].set_xlabel('Age')
axes[0, 1].set_ylabel('Hits per At-Bat')

# Plot 3: Career trajectory
axes[1, 0].plot(mlb_df['age'], mlb_df['hits_per_ab'], marker='o', linewidth=2, markersize=8)
axes[1, 0].set_xlabel('Age')
axes[1, 0].set_ylabel('Hits per At-Bat')
axes[1, 0].set_title('Career Performance Trajectory')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Performance by year
axes[1, 1].plot(mlb_df['year'], mlb_df['hits_per_ab'], marker='s', linewidth=2, markersize=6)
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Hits per At-Bat')
axes[1, 1].set_title('Performance by Season')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Statistical analysis
print("\nStatistical Analysis:")
correlation = mlb_df['age'].corr(mlb_df['hits_per_ab'])
print(f"Correlation between age and hits/AB: {correlation:.3f}")

# Find peak performance age
avg_by_age = mlb_df.groupby('age')['hits_per_ab'].mean().sort_values(ascending=False)
print(f"\nAverage performance by age (top 5):")
print(avg_by_age.head())

## Key Findings: MLB Analysis

Based on our analysis of hits per at-bat vs age for MLB players (2000-2015 era):

1. **Peak Performance**: Identify the age range where players typically achieve their highest batting performance
2. **Career Trajectory**: Observe how performance changes throughout a player's career
3. **Decline Patterns**: Analyze when and how performance typically declines

## Next Steps

This analysis will be extended to include:
- More MLB players and different performance metrics
- NFL analysis (yards per game, touchdowns, etc.)
- NBA analysis (points per game, efficiency metrics)
- NHL analysis (goals, assists, points per game)
- Cross-sport comparison of prime performance ages

In [ ]:
# Save processed data for future analysis
import os

# Create data directory if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

# Save the DataFrame
mlb_df.to_csv('data/mlb_sample_analysis.csv', index=False)
print("Data saved to 'data/mlb_sample_analysis.csv'")

# Display final summary
print("\n" + "="*50)
print("SPORTS PRIMES ANALYSIS - MLB COMPONENT COMPLETE")
print("="*50)
print(f"✓ Analyzed {len(mlb_df)} player-seasons")
print(f"✓ Age range: {mlb_df['age'].min()}-{mlb_df['age'].max()} years")
print(f"✓ Performance metric: Hits per At-Bat")
print(f"✓ Peak performance age identified")
print(f"✓ Data visualization complete")
print(f"✓ Results saved for cross-sport comparison")
print("\nReady for expansion to NFL, NBA, and NHL analysis!")